In [1]:
import requests
import json
import pandas as pd
import math
import os
from IPython.display import Image, display, clear_output


In [2]:
def readJson(file):
    with open(file,"r") as f:
        rf=json.load(f)
    return rf

def readTsv(file):
    with open(file, "r") as f:
        rf=csv.DictReader(f, dialect='excel-tab')
    return rf

In [3]:
# Initial requests from API url (23.08.19)
config = readJson("config.json")
client_id=config["client_id"]
elev_key = config["elevation_key"]
test_area = config["test_area"]

def requestImagesBySequence(sequence_key):
    request = requests.get("https://a.mapillary.com/v3/images/?sequence_keys="+sequence_key+"&per_page=10000&client_id="+client_id)
    return request.json()

In [4]:
# data = readJson("mapillary_data.json")
data = pd.read_csv("training_data/features_no_dupes.csv").drop_duplicates(subset="id",keep="first").set_index("id")
data

,avg_speed,min_speed,max_speed,max_accel,max_deaccel,mean_dist_med_speed,sd_speed,percent_ped_speed,percent_bic_speed,percent_car_speed,num_stops,distance,nsd,sinuosity,turosity,convex_hull_area,mean_relative_angle,duration
id,,,,,,,,,,,,,,,,,,
6lmxhlsdu2820pp9ewk41z,16.092218,0.232964,25.961550,10.652202,-13.501058,12.864293,5.964283,0.697674,7.209302,86.511628,0,7917.451715,87.347028,1.037741,0.004473,0.155636,0.054281,492.005
g234mtreigui4a64i4xea9,6.322339,0.141193,12.872192,3.198201,-6.585177,6.365499,2.824493,0.735294,26.470588,36.764706,2,1132.324604,33.529708,1.007190,0.012726,0.021988,0.142937,179.099
hwt93drmgmyk3pnt7hib08,1.275309,0.620147,2.196088,1.129826,-7.976933,0.787971,0.274747,67.193676,0.000000,0.000000,0,366.273853,12.807824,2.232828,0.009084,0.006280,-0.585815,287.204
9gmnn1cdfv7vbptl9p83u2,1.210824,0.079203,2.395365,1.275051,-4.657094,1.158081,0.336322,60.103627,0.000000,0.000000,4,562.095759,20.756349,1.304693,0.006812,0.013878,0.552962,464.226
18ejk5cjxixj051qiupxyb,1.313297,0.637032,2.017232,1.256552,-2.369025,0.690100,0.291555,57.831325,0.000000,0.000000,0,126.595226,11.028732,1.040797,0.024806,0.003729,0.468846,96.395
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
frlAsAiloH7sIbIHy4ZyAg,1.484849,1.100335,2.089263,1.990085,-0.470809,0.494464,0.299528,38.095238,0.000000,0.000000,0,233.121253,15.262202,1.000800,0.185803,0.004288,0.086987,157.000
KbVGvM_dpBZidNPlXwOMnQ,1.321091,0.869768,1.805226,1.448281,-0.345011,0.467729,0.240208,64.705882,0.000000,0.000000,0,158.530978,12.202305,1.064708,0.141979,0.004055,1.318120,120.000
k9FvSmiv2eRKtfCP48gGzA,1.237887,0.305195,2.456294,2.003500,-0.292721,1.075550,0.448620,39.534884,0.000000,0.000000,0,269.859348,16.402692,1.003015,0.036409,0.004943,0.228754,218.000


In [49]:
write = "sequence_labels_1.csv"
skipped = "skipped_segments_1.csv"
f = open(write, "a+")
fs = open(skipped,"a+")
num_labelled = pd.read_csv(write, header=None).shape[0] if os.stat(write).st_size != 0 else 0
num_skipped = pd.read_csv(skipped, header=None).shape[0] if os.stat(skipped).st_size != 0 else 0
index_to_start = num_labelled+num_skipped
a = ""
num = data.shape[0]
num_imgs = 10


for i, seq in enumerate(data.index[index_to_start:], index_to_start):
    images = list(reversed(requestImagesBySequence(seq)["features"]))
    interval = math.ceil(len(images)/num_imgs) if len(images)<50 else 10
    if(a=="q"):
        break
    j = 0
    incr = 1
    while(True):
        if(len(images)<30):
            break
        clear_output()
        print(f"[INFO]\nprogress:{i}/{num}\tnum_labelled:{num_labelled}\n")
        print(j+interval, len(images))
        key = images[j]["properties"]["key"]
        display(Image(url= f"https://images.mapillary.com/{key}/thumb-320.jpg"))
        a = input(f"0 = ped, 1 = bic, 2 = car, q=quit, anything_else = next image\n{data.iloc[i]}\n")
        if(a=="q"):
            break
        elif(len(a)>0 and a=="s"):
            fs.write(f"{seq}\n")
            break
        elif(len(a)>0 and a=="i"):
            if(interval==math.ceil(len(images)/num_imgs) or interval==10):
                interval=1
            else:
                interval = math.ceil(len(images)/num_imgs) if len(images)<50 else 10
        elif(len(a)>0 and a=="f"):
            images = list(reversed(images))
            j = 0
        elif(len(a)>0 and (int(a)==0 or int(a)==1 or int(a)==2)):
            f.write(f"{a},{seq}\n")
            num_labelled+=1
            break
        else: 
            if(j+interval>len(images)-1):
                if(incr==len(images)-1):
                    j = 0
                    incr = 1
                else:
                    j = incr
                    incr+=1
                print("More images")
            else:
                j+=interval

f.close()
fs.close()

[INFO]
progress:1195/1863	num_labelled:963

10 346


KeyboardInterrupt: 